In [24]:
import pandas as pd
import ee
import geetools
import folium
import time
from folium import plugins
import Leaf_tools
import math
from collections import defaultdict
import numpy as np
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AY0e-g6HolP5_0yf_ZICtvMJOcO9x8HFl2yArwF-CRXJqY1x_J16Y86xrgg



Successfully saved authorization token.


**Import data**

In [25]:
tiles_CCRS = ee.FeatureCollection("users/ganghong/TilesSampling_Sub")
print (tiles_CCRS.size().getInfo())
tiles_Bel = ee.FeatureCollection('users/ganghong/TilesSampling_Belmanip_Sub')
print (tiles_Bel.size().getInfo())
ezone = ee.FeatureCollection("users/ganghong/NA_Forests")
fapar_CCRS = ee.ImageCollection("users/ganghong/FAPAR_CCRS")
fcover_CCRS = ee.ImageCollection("users/ganghong/FCOVER_CCRS")
LAI_CCRS = ee.ImageCollection("users/ganghong/LAI_CCRS")
fapar_Bel = ee.ImageCollection("users/ganghong/FAPAR_Bel")
fcover_Bel = ee.ImageCollection("users/ganghong/FCOVER_Bel")
LAI_Bel = ee.ImageCollection("users/ganghong/LAI_Bel")
##Define Forest Cover map to use - should be a global constant
NALCMS_forest = ee.ImageCollection('users/rfernand387/NA_NALCMS_2015_tiles')\
.map(lambda image:image.select("b1").rename("landcover"))\
.merge(ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V/Global")\
.map(lambda image:image.select("discrete_classification").remap([0,20,30,40,50,60,70,80,90,100,111,112,113,114,115,116,121,122,123,124,125,126,200],[0,8,10,15,17,16,19,18,14,13,1,3,1,5,6,6,2,4,2,5,6,6,18],0).toUint8().rename("landcover")))

# print (fapar_CCRS.size().getInfo())
# print (fcover_CCRS.size().getInfo())
# print (LAI_CCRS.size().getInfo())
# print (fapar_Bel.size().getInfo())
# print (fcover_Bel.size().getInfo())
# print (LAI_Bel.size().getInfo())

28
35


**Functions definition**

In [26]:
basemaps = {
'Google Maps': folium.TileLayer(
tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
attr = 'Google',
name = 'Google Maps',
overlay = True,
control = True
),
'Google Satellite': folium.TileLayer(
tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
attr = 'Google',
name = 'Google Satellite',
overlay = True,
control = True
)}

## display layers
def add_ee_layer(self, ee_object, vis_params, name):
 try:
  if isinstance(ee_object, ee.image.Image):
   map_id_dict = ee.Image(ee_object).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object, ee.imagecollection.ImageCollection):
   ee_object_new = ee_object.mosaic()
   map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name, 
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object, ee.geometry.Geometry):
   folium.GeoJson(
   data = ee_object.getInfo(),
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object,ee.featurecollection.FeatureCollection):
   ee_object_new = ee.Image().paint(ee_object, 0, 2)
   map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

 except:
  print("Could not display {}".format(name))

#date conversion
def DatetoSystemtime(datelist):
    geestamp=[]
    for date in datelist:       
        date = pd.to_datetime(date, format='%Y-%m-%d')       
       # print (date)
        year_day = pd.Timestamp(year=date.year, month=1, day=1)       
        doy = (date - year_day).days + 1     ## add three digits ###   
        doy='{:>03}'.format(doy) ### three digit format for doy 000
        hour = '12'
        year=date.year
        timestamp = str(year) + doy + hour
        # convert time stamp to gee format
        gee_timestamp = int(pd.to_datetime(timestamp,format='%Y%j%H').value // 10**6)
        #print (gee_timestamp)
        geestamp.append(gee_timestamp)        
    return geestamp    

#remove properties 
def removeProperty(feat): 
    properties = feat.propertyNames()
    selectProperties = properties.filter(ee.Filter.eq('item', 'MGRS_TILE')) 
    return feat.select(selectProperties)

#featurecollection intersection
def featurecollection_interset(tile, resultlist):
    resultlist = ee.List(resultlist)
    tile = ee.Feature(tile)
    def polygonintersect(zone):    
        zone = ee.Feature(zone)   
        intersection = tile.intersection(zone, ee.ErrorMargin(1));
        return intersection.set({'ezone': zone.get('EZMAP_ZONE')}).set({'Intersect': intersection.area().divide(1000 * 1000)})
    intersection = zoneList.map(polygonintersect)
    sortedintersect=ee.FeatureCollection(intersection).sort('Intersect', False)
    return resultlist.add(sortedintersect.first())

#land cover frequency vector based
def calculateFreq(tile,list):
    list=ee.List(list)
    frequency = NALCMS_forest.mosaic().reduceRegion(
       reducer=ee.Reducer.frequencyHistogram(),
       maxPixels=2e9,
       scale=500,
       geometry=tile.geometry()
      )
    tempobj= frequency.get("landcover")
    okeys = ee.Dictionary(tempobj).keys()
    ovals = ee.Dictionary(tempobj).values()
    minmax = ee.Dictionary(ovals.reduce(ee.Reducer.minMax()))
    maxval = ee.Number(minmax.get('max'))
    maxind = ovals.indexOf(maxval)
    maxkey = okeys.get(maxind)
    tile= tile.set({'landcover': maxkey})
    return list.add(tile)

#feature to dataframe
def fc2df(fc):
    # Convert a FeatureCollection into a pandas DataFrame
    # Features is a list of dict with the output
    features = fc.getInfo()['features']
    dictarr = []
    for f in features:
        # Store all attributes in a dict
        attr = f['properties']
        # and treat geometry separately
        #attr['geometry'] = f['geometry']  # GeoJSON Feature!
        # attr['geometrytype'] = f['geometry']['type']
        dictarr.append(attr)
        df = GeoDataFrame(dictarr)
        # Convert GeoJSON features to shape
        #df['geometry'] = map(lambda s: shape(s), df.geometry)
    return df

#add date to a feature
def datelist(img):
    imgdate=ee.Date(img.get("system:time_start"))
    return ee.Feature(None, {'date': imgdate.format('YYYY-MM-dd')})
 
#bit wsie operation  
def bitwiseExtract(value, fromBit, toBit):
    maskSize = ee.Number(1).add(toBit).subtract(fromBit)
    mask = ee.Number(1).leftShift(maskSize).subtract(1)
    return value.rightShift(fromBit).bitwiseAnd(mask)

#modis product bit mask
def bitmask(image):
    qa = image.select('FparLai_QC')
    good = bitwiseExtract(qa, 0,0) # returns 0 for good quality
    return image.updateMask(ee.Image(good).eq(0))  ## needs to be inverted to mask pixels with bad quality, 0 means good quality , pass the mask  

## function for calculate raster dominant 
def dominantCal(raster, coarseProj, coarseScale, targetscale, minVal, maxVal, steps):
    ##calcualte stats from reduceResolution
    stats = raster.reproject(raster.projection(), None, targetscale).reduceResolution(
       reducer= ee.Reducer.fixedHistogram(minVal, maxVal, steps),
       maxPixels= 65535,
       bestEffort= True
    ).reproject(
      crs=coarseProj,
      scale= coarseScale##MODISscale
    )

    array=stats.toArray()
    ## slice to land cover
    landcover=array.arraySlice(1,0,1)
    ##slice to percentage
    percentage=array.arraySlice(1,1,2)
    ##sort percentage
    sortedper = array.arraySort(percentage);
    ## get values
    length = sortedper.arrayLength(0)
    values = sortedper.arraySlice(0, length.subtract(1), length);
    ##convert back to an image  
    maxpercentage = values.arrayProject([1]).arrayFlatten([['landcover', 'percentage']])
    dominantlc=maxpercentage.select('landcover').ceil()
    return maxpercentage.select('landcover').ceil()

## function for add dominat lc and ez
def addDZ(dominantEZ,dominantLC):
    def wrap(img):
        return img.addBands(dominantEZ.rename('dominantEZ')).addBands(dominantLC.rename('dominantLC'))
    return wrap

 ## Extract the ID of each image 
def  getidList(item):
     return ee.Image(item).id()
 
 ## matched date between sensors   
def datematched(halftempRes,sensorname, colCoarse):
    def wrap(img):
            dateImg=img.get('date')
            startdate= ee.Date(dateImg).advance(-halftempRes,'day')  
            enddate= ee.Date(dateImg).advance(halftempRes,'day')
            #size = colCoarse.filterDate(startdate, enddate).size()  ## dataMODIS is a default reference sensor    
            colCoarse2 = colCoarse.filterDate(startdate, enddate)  ## dataMODIS is a default reference sensor
            size=colCoarse2.size()
            return img.set('matched'+sensorname,ee.Algorithms.If(size.eq(0),'none', colCoarse2.toList(size)))  ## change matcheddata to matcheddata plus sensor
    return wrap

## mask for CLGS data  CLGS_flag
def validCGLSMask(CGLS_flag):
    def wrap(image):
        qa = image.select(CGLS_flag)   ## CGLS has different bands for each parameter, like b3 for LAI, b2 for FAPAR
        land = qa.bitwiseAnd(1 << 0)   ## bit 0 equal to 1, land 
        #return image.unmask().updateMask(land)  ##bit 0 equal to 1 means land ; Goolge Earth Engine masked no-data already
        return image.updateMask(land)  ##bit 0 equal to 1 means land ; Goolge Earth Engine masked no-data already
    return wrap

## mask for MODIS data
def validMODISMask(image):
    qa = image.select('FparLai_QC')
    qaBitsGood = qa.bitwiseAnd(1 << 0) ##bit 0, 0 mean good , 1 bad pixel
    validmask=qaBitsGood.eq(0) 
    return image.updateMask(validmask)


In [27]:
##the function to get the corase resoltuion image collection with matched sensor information 
## startDate, endDate: dates to start and end
##mapBounds: tile boundary 
## tilename: MGRS tile used to filter data, used in S2 only
##parameter: LAI, fAPAR,fCOVER,make sure correponding to name definition leaf-tool box
##tempRes: temporal resolution, like 5 days, 7 days
##coarsescale: scale used for getting the coarse resolution
## MODIS_paraBand, MODIS_error: names of MODIS parameter and uncertainty bands
## S2_paraBand, S2_error: names of S2 parameter and uncertianty bands
## CGLS_parameter: name of parameter Collection, LAI_CCRS in the import data section
## CGLS_paraBand, CGLS_error, CGLS_flag: names of CGLS parameter , uncertianty and quality flag bands
def applySensorComp(startDate, endDate, mapBounds, tilename, parameter, tempRes, coarsecollection, coarsescale, MODIS_paraBand, MODIS_error, S2_paraBand, S2_error, CGLS_parameter, CGLS_paraBand, CGLS_error, CGLS_flag):
      
    para_error=parameter+'error' ## 'LAIerror'
     ### Subsetting image collections,  property setting, flag mask updating
    ##update mask and set a new property date
    dataMODIS =ee.ImageCollection('MODIS/006/MCD15A3H')\
                      .filterBounds(mapBounds)\
                      .filterDate(starDate, endDate)\
                       .map(validMODISMask)\
                       .map(lambda img:img.select([MODIS_paraBand, MODIS_error],[parameter, para_error]).set('date', ee.Date(img.get('system:time_start')).format("yyyy-MM-dd")).clip(mapBounds))   #['Lai','LaiStdDev'],['LAI','LAIun']
    
    dataCGLS=CGLS_parameter.filterBounds(mapBounds)\
                           .filterDate(starDate, endDate)\
                           .map(validCGLSMask(CGLS_flag))\
                           .map(lambda img:img.select([CGLS_paraBand,CGLS_error],[parameter, para_error]).set('date', ee.Date(img.get('system:time_start')).format("yyyy-MM-dd")).clip(mapBounds)) 
   
    dataS2=Leaf_tools.Leaf_tools(mapBounds,starDate,endDate,tilename,parameter)\
                     .map(lambda img:img.updateMask(img.select('QC').eq(0)))\
                     .map(lambda img:img.select([S2_paraBand,S2_error],[parameter, para_error]).set('date', ee.Date(img.get('system:time_start')).format("yyyy-MM-dd")).clip(mapBounds))
    
    ### date matching    
    halftempRes=math.ceil(tempRes/2)
    ##print (halftempRes)
   # dataS2matched= dataS2.map(datematched(halftempRes,'MODIS',dataMODIS))    
   # dataCGLSmatched= dataCGLS.map(datematched(halftempRes,'MODIS',dataMODIS))
    dataS2matched= dataS2.map(datematched(halftempRes,'MODIS',dataMODIS)).map(datematched(halftempRes,'CGLS',dataCGLS))
    
    ### coarse resolution reducers  
    reducers = ee.Reducer.mean().combine(
      reducer2= ee.Reducer.stdDev(),
      sharedInputs= True
    ).combine(
      reducer2= ee.Reducer.count(),
      sharedInputs= True
    )
    coarsecrs=coarsecollection.first().projection()
    ##coarse CGLS-product1
    coarseCGLS=dataCGLS.map(lambda img:img.reproject(dataCGLS.first().projection(), None, 300).reduceResolution(
               reducer= reducers,
               maxPixels= 65535,
               bestEffort= True
               ).reproject(
                   crs= coarsecrs,
                   scale= coarsescale
                 ))
    ##coarse S2-product2
    coarseS2=dataS2matched.map(lambda img:img.reproject(dataS2matched.first().projection(), None, 20).reduceResolution(
               reducer= reducers,
               maxPixels= 65535,
               bestEffort= True
               ).reproject(
                   crs= coarsecrs,
                   scale= coarsescale
                 ))
    #### coarse MODIS product
    coarseMODIS=dataMODIS.map(lambda img:img.reproject(dataMODIS.first().projection(), None, 500).reduceResolution(
               reducer= reducers,
               maxPixels= 65535,
               bestEffort= True
               ).reproject(
                   crs= coarsecrs,
                   scale= coarsescale
                 ))
    
    ### dominant land cover
    tilelandcover=NALCMS_forest.filterBounds(mapBounds).mosaic().clip(mapBounds)  
    dominantLC=dominantCal(tilelandcover, coarsecrs,  coarsescale, 90, 0, 19, 19) ##(0, 19, 19)
    
    ## convert ezone ploygon to raster
    ezraster=ee.FeatureCollection(ezone).reduceToImage(['EZMAP_CODE'],ee.Reducer.first( )).clip(mapBounds)
    tileez=ezraster.clip(mapBounds)
     ### dominant ezone
    dominantEZ=dominantCal(tileez, coarsecrs, coarsescale, 200, 0, 99, 99) ##(0, 99, 99)
 
    ## add dominant land cover,ezone
    coarseCGLSwithDominant=coarseCGLS.map(addDZ(dominantEZ,dominantLC))   
     ## add dominant land cover,ezone
    coarseS2withDominant=coarseS2.map(addDZ(dominantEZ,dominantLC))
    ## add dominant land cover,ezone
    coarseMODISwithDominant=coarseMODIS.map(addDZ(dominantEZ,dominantLC))
    
    ### return three coarse image collection, the matched MODIS data information include the property matchedData, 
    ###but the images information included is still orignal MODIS images, 
    ##it could be updated through getting system id from original MODIS and use ee.Filter.inList to find the corresponding images in coarse MODIS collection.    
    return coarseS2withDominant, coarseCGLSwithDominant,coarseMODISwithDominant  

In [28]:
## convert GEE image to numpy array
def imgTonumpy(band_arrs):
     # # Get individual band arrays.
    band_LAImean = band_arrs.get('LAI_mean')
    band_LAIstdDev = band_arrs.get('LAI_stdDev')
    band_LAIcount = band_arrs.get('LAI_count')
    band_LAIerrormean = band_arrs.get('LAIerror_mean')
    band_LAIerrorcount = band_arrs.get('LAIerror_count')
   # band_dominantEZ = band_arrs.get('dominantEZ')
   # band_dominantLC = band_arrs.get('dominantLC')
    # # Transfer the arrays from server to client and cast as np array.
    np_LAImean = np.array(band_LAImean.getInfo())
    np_LAIstdDev = np.array(band_LAIstdDev.getInfo())
    np_LAIcount = np.array(band_LAIcount.getInfo())
    np_LAIerrormean = np.array(band_LAIerrormean.getInfo())
    np_LAIerrorcount = np.array(band_LAIerrorcount.getInfo())
    #np_dominantEZ = np.array(band_dominantEZ.getInfo())
    #np_dominantLC = np.array(band_dominantLC.getInfo())
    # # Expand the dimensions of the images so they can be concatenated into 3-D.
    np_LAImean = np.expand_dims(np_LAImean, 2)
    np_LAIstdDev = np.expand_dims(np_LAIstdDev, 2)
    np_LAIcount = np.expand_dims(np_LAIcount, 2)
    np_LAIerrormean = np.expand_dims(np_LAIerrormean, 2)
    np_LAIerrorcount = np.expand_dims(np_LAIerrorcount, 2)
    #np_dominantEZ = np.expand_dims(np_dominantEZ, 2)
    #np_dominantLC = np.expand_dims(np_dominantLC, 2)
        # # Stack the individual bands to make a 3-D array.
    img_stack = np.concatenate((np_LAImean, np_LAIstdDev, np_LAIcount, np_LAIerrormean, np_LAIerrorcount), 2)
#     img_stack = np_LAImean
    #img_stack = np.concatenate((np_LAImean, np_LAIstdDev, np_LAIcount, np_LAIerrormean, np_LAIerrorcount,np_dominantEZ, np_dominantLC), 2)
        
    return img_stack

In [31]:
## convert GEE images to a dictionary list
def imageToarray(aoi, coarseS2, coarseCGLS, coarseMODIS):

    imgList_S2 = coarseS2.toList(coarseS2.size())  # convert collection to list
    print('size of imageList_S2:  ', imgList_S2.size().getInfo())

    sensorList = []  # sensor list: ouput of this functin:  element is dictionary, S2: array, MODIS: array/'none', CGLS: array/'none'
    images_dict = defaultdict(list)

    for i in range(coarseS2.size().getInfo()):
        ## get image from image list
        image_S2 = ee.Image(imgList_S2.get(i)).select(['LAI_mean', 'LAI_stdDev', 'LAI_count','LAIerror_mean', 'LAIerror_count','dominantEZ','dominantLC'])
        #print (image_S2.getInfo()['bands'][0])
        #print (image_S2.propertyNames().getInfo())
        # Get 2-d pixel array for AOI - returns feature with 2-D pixel array as property per band.
        band_S2_arrs = image_S2.sampleRectangle(region=aoi, defaultValue=255)
       # print(band_S2_arrs.getInfo())
       # S2_img=imgTonumpy(band_S2_arrs)
       # print(S2_img.shape)
       # images_dict['S2'].append(S2_img)
        images_dict['S2']='S2'

        ### extract images from MODIS
        MODISlist_S2 = image_S2.get('matchedMODIS')  ## get the property 'matchedMODIS' of  the image 
        if (MODISlist_S2.getInfo() != 'none'):
            MODISlist_S2 = ee.List(MODISlist_S2).map(getidList)  ## get the ids of images in the property 'matchedData'
        ### get the corresponding coarse MODIS
        ## use the sytem index to locate coarse MODIS since the matching was occured before the coarse resolution, the image in the matched property is original images
            coarseMODIS_S2 = coarseMODIS.filter(ee.Filter.inList('system:index', MODISlist_S2))
            ### convert coarse MODIS to List
            imgList_MODIS = coarseMODIS_S2.toList(coarseMODIS_S2.size())  # convert collection to list
            for j in range(coarseMODIS_S2.size().getInfo()):
                # get image from MODIS list
                image_MODIS = ee.Image(imgList_MODIS.get(j)).select(['LAI_mean', 'LAI_stdDev', 'LAI_count','LAIerror_mean', 'LAIerror_count','dominantEZ','dominantLC'])##.unmask()
                # Get 2-d pixel array for AOI - returns feature with 2-D pixel array as property per band.
                band_arrs_MODIS = image_MODIS.sampleRectangle(region=aoi, defaultValue=255)
                #print (band_arrs_MODIS.getInfo())              
                MODIS_img=imgTonumpy(band_arrs_MODIS)            
                print('MODIS',MODIS_img.shape)
                images_dict['MODIS'].append(MODIS_img)
                #images_dict['MODIS'] = 'MODIS'
        else:
            images_dict['MODIS'] = 'none'

        ### extract images from CGLS
        CGLSlist_S2 = image_S2.get('matchedCGLS')  ## get the property 'matchedData' of  first image of the list
        if (CGLSlist_S2.getInfo() != 'none'):
            CGLSlist_S2 = ee.List(CGLSlist_S2).map(getidList)  ## get the ids of images in the property 'matchedData'
        ### get the corresponding coarse MODIS
        ## use the sytem index to locate coarse MODIS since the matching was occured before the coarse resolution, the image in the matched property is original images
            coarseCGLS_S2 = coarseCGLS.filter(ee.Filter.inList('system:index', CGLSlist_S2))
            ### convert coarse MODIS to List
            imgList_CGLS = coarseCGLS_S2.toList(coarseCGLS_S2.size())  # convert collection to list

            for k in range(coarseCGLS_S2.size().getInfo()):
                image_CGLS = ee.Image(imgList_CGLS.get(k)).select(['LAI_mean', 'LAI_stdDev', 'LAI_count','LAIerror_mean', 'LAIerror_count','dominantEZ','dominantLC'])##.unmask()
                # Get 2-d pixel array for AOI - returns feature with 2-D pixel array as property per band.
                band_arrs_CGLS = image_CGLS.sampleRectangle(region=aoi, defaultValue=255)
                #print (band_arrs_CGLS.getInfo())               
                CGLS_img=imgTonumpy(band_arrs_CGLS)            
                print('CGLS',CGLS_img.shape)
                ### add image to dictionary
                images_dict['CGLS'].append(CGLS_img)
                #images_dict['CGLS'] = 'CGLS'
        else:
            images_dict['CGLS'] = 'none'

        ### add dictionary to list
        sensorList.append(images_dict.copy())
        images_dict.clear() ## clear values of dictionary

    return sensorList

**test Output**

In [25]:
starDate='2019-04-01'
endDate='2019-06-30'
mapBounds= ee.FeatureCollection(ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(0))).geometry()
tilename=ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(0)).get('MGRS_TILE')
print('MGRS tilename:',tilename.getInfo())
## run function applySensorComp
dataMODIS2 =ee.ImageCollection('MODIS/006/MCD15A3H')\
                      .filterBounds(mapBounds)\
                      .filterDate(starDate, endDate)
coarseS2_output, coarseCGLS_output,coarseMODIS_output=applySensorComp(starDate, endDate, mapBounds, tilename, 'LAI', 5,dataMODIS2, 500, 'Lai','LaiStdDev', 'estimateLAI','errorLAI', LAI_CCRS,'b1','b2','b3')
print (coarseS2_output.size().getInfo(), 'coarseS2_output')
print (coarseCGLS_output.size().getInfo(), 'coarseCGLS_output')
print (coarseMODIS_output.size().getInfo(), 'coarseMODIS_output')

print ("########################################DEMO for extract matched images between sensors##################################")
print ("########################################S2################################################################################")
imgList_S2dominant = coarseS2_output.toList(coarseS2_output.size()) # convert colleciton to list
print('size of imageList_S2:  ', imgList_S2dominant.size().getInfo());
props_S2dominant = ee.Image(imgList_S2dominant.get(5)).toDictionary(ee.Image(imgList_S2dominant.get(5)).propertyNames())
#print (props_S2dominant.getInfo())
bands_S2dominant = ee.Image(imgList_S2dominant.get(5)).bandNames() ## get the band names of  first image of the list
print ('band names:  ',bands_S2dominant.getInfo())
print ('S2 index 5:  ', ee.Image(imgList_S2dominant.get(5)).getInfo())
#print (imgList_S2dominant.get(5).getInfo()['bands'][0]['dimensions'])
#print (imgList_S2dominant.get(5).getInfo()['bands'][0])
print ("########################################CGLS##############################################################################")
CGLSlist_S2dominant=ee.Image(imgList_S2dominant.get(5)).get('matchedCGLS') ## get the property 'matchedCGLS' of  first image of the list
print ('matched CGLS_S2:  ', CGLSlist_S2dominant.getInfo())
print ('size of matched CGLS_S2:  ',ee.List(CGLSlist_S2dominant).size().getInfo())  # size of the property 'matchedCGLS' 
#print (imageList.get(0), 'Result index 0 ')
#print (ee.Image(imageList.get(0)).get('matchedData'), 'matched MODIS')

CGLSlist_S2=ee.List(CGLSlist_S2dominant).map(getidList) ## get the ids of images in the property 'matchedData'
print ('CGLSlist_S2 sytem id list:  ',CGLSlist_S2.getInfo())
### get the corresponding coarse MODIS
## use the sytem index to locate coarse MODIS since the matching was occured before the coarse resolution, the image in the matched property is original images
coarseCGLSList_S2= coarseCGLS_output.filter(ee.Filter.inList('system:index', CGLSlist_S2)) 
print ('coarseCGLSList_S2 band name:',ee.Image(coarseCGLSList_S2.toList(coarseCGLSList_S2.size()).get(0)).bandNames().getInfo())  
print ('coarseCGLSList_S2 image index 0:',ee.Image(coarseCGLSList_S2.toList(coarseCGLSList_S2.size()).get(0)).getInfo()) 
print ("########################################MODIS##############################################################################")
MODISlist_S2dominant=ee.Image(imgList_S2dominant.get(5)).get('matchedMODIS') ## get the property 'matchedMODIS' of  first image of the list
print ('matched MODIS_S2:  ', MODISlist_S2dominant.getInfo())
print ('size of matched MODIS_S2:  ',ee.List(MODISlist_S2dominant).size().getInfo())  # size of the property 'matchedMODIS' 

MODISlist_S2=ee.List(MODISlist_S2dominant).map(getidList) ## get the ids of images in the property 'matchedData'
print ('MODISlist_S2 sytem id list:  ',MODISlist_S2.getInfo())
### get the corresponding coarse MODIS
## use the sytem index to locate coarse MODIS since the matching was occured before the coarse resolution, the image in the matched property is original images
coarseMODISList_S2= coarseMODIS_output.filter(ee.Filter.inList('system:index', MODISlist_S2)) 
print ('coarseMODISList_S2 band name:',ee.Image(coarseMODISList_S2.toList(coarseMODISList_S2.size()).get(0)).bandNames().getInfo())  
print ('coarseMODISList_S2 image index 0:',ee.Image(coarseMODISList_S2.toList(coarseMODISList_S2.size()).get(0)).getInfo()) 

MGRS tilename: 09WXP
49 coarseS2_output
8 coarseCGLS_output
22 coarseMODIS_output
########################################DEMO for extract matched images between sensors##################################
########################################S2################################################################################
size of imageList_S2:   49
band names:   ['LAI_mean', 'LAI_stdDev', 'LAI_count', 'LAIerror_mean', 'LAIerror_stdDev', 'LAIerror_count', 'dominantEZ', 'dominantLC']
S2 index 5:   {'type': 'Image', 'bands': [{'id': 'LAI_mean', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'SR-ORG:6974', 'crs_transform': [500.00000000053956, 0, -20015109.354, 0, 499.9999999994604, 10007554.677]}, {'id': 'LAI_stdDev', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'SR-ORG:6974', 'crs_transform': [500.00000000053956, 0, -20015109.354, 0, 499.9999999994604, 10007554.677]}, {'id': 'LAI_count', 'data_type': {'type': 'PixelType', 'precision': 'int', 'mi

**Run functions**

In [32]:
starDate='2019-04-01'
endDate='2019-04-15'
mapBounds= ee.FeatureCollection(ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(0))).geometry()
print (mapBounds.bounds().getInfo())
tilename=ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(0)).get('MGRS_TILE')
print('MGRS tilename:',tilename.getInfo())
## run function applySensorComp
dataMODIS2 =ee.ImageCollection('MODIS/006/MCD15A3H')\
                      .filterBounds(mapBounds)\
                      .filterDate(starDate, endDate)
coarseS2_output, coarseCGLS_output,coarseMODIS_output=applySensorComp(starDate, endDate, mapBounds, tilename, 'LAI', 5, dataMODIS2, 500, 'Lai','LaiStdDev', 'estimateLAI','errorLAI', LAI_CCRS,'b1','b2','b3')
print (coarseS2_output.size().getInfo(), 'coarseS2_output')
print (coarseCGLS_output.size().getInfo(), 'coarseCGLS_output')
print (coarseMODIS_output.size().getInfo(), 'coarseMODIS_output')

coarsecrs=coarseS2_output.first().projection()
coarsescale=coarseS2_output.first().projection().nominalScale()
print (coarsecrs.getInfo())
print (coarsescale.getInfo())
print (mapBounds.projection().getInfo())
print ("########################################DEMO for extract matched images between sensors##################################")
arraynp_output=imageToarray(mapBounds.bounds(), coarseS2_output, coarseCGLS_output, coarseMODIS_output)
print (len(arraynp_output))
print (arraynp_output[5])

{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-126.8205843103241, 65.66551559023394], [-124.25347520134201, 65.66551559023394], [-124.25347520134201, 66.70265632676876], [-126.8205843103241, 66.70265632676876], [-126.8205843103241, 65.66551559023394]]]}
MGRS tilename: 09WXP
8 coarseS2_output
1 coarseCGLS_output
3 coarseMODIS_output
{'type': 'Projection', 'crs': 'SR-ORG:6974', 'transform': [500.00000000053956, 0, -20015109.354, 0, 499.9999999994604, 10007554.677]}
499.99999999999994
{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}
########################################DEMO for extract matched images between sensors##################################
size of imageList_S2:   8
MODIS (231, 694, 5)
MODIS (231, 694, 5)
MODIS (231, 694, 5)
MODIS (231, 694, 5)
MODIS (231, 694, 5)
MODIS (231, 694, 5)
CGLS (231, 694, 5)
MODIS (231, 694, 5)
CGLS (231, 694, 5)
MODIS (231, 694, 5)
CGLS (231, 694, 5)
MODIS (231, 694, 5)
8
defaultdict(<class 'list'>, {'S2': 'S2'

**test Fapar**

In [9]:
coarseS2_Fapar_output, coarseCGLS_Fapar_ouput,coarseMODIS_Fapar_output=applySensorComp('2019-04-01', '2019-09-30', mapBounds, tilename, 'fAPAR', 5, dataMODIS2, 500,'Fpar','FparStdDev', 'estimatefAPAR','errorfAPAR', fapar_CCRS,'b6','b1','b2')
print (coarseS2_Fapar_output.size().getInfo(), 'coarseS2_Fapar_output')
print (coarseCGLS_Fapar_ouput.size().getInfo(), 'coarseCGLS_Fapar_ouput')
print (coarseMODIS_Fapar_output.size().getInfo(), 'coarseMODIS_Fapar_output')

85 coarseS2_Fapar_output
17 coarseCGLS_Fapar_ouput
45 coarseMODIS_Fapar_output
